# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

In [3]:
region_one = pd.read_csv('/datasets/geo_data_0.csv')
region_two = pd.read_csv('/datasets/geo_data_1.csv')
region_three = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
region_one.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [18]:
def information(region):
    display('Первые пять строк:')
    display(region.head())
    display('-------------------------------------------------------------------------------------------------------------')
    display('Общая информация:') 
    display(region.info())
    display('-------------------------------------------------------------------------------------------------------------')
    display('Описание:') 
    display(region.describe())
    display('-------------------------------------------------------------------------------------------------------------')
    display('Три случайные строки:') 
    display(region.sample(n=3))
    display('-------------------------------------------------------------------------------------------------------------')
    display('Количество дубликатов:') 
    display(region.duplicated().sum())

### Первое месторождение:

In [19]:
information(region_one)

'Первые пять строк:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'-------------------------------------------------------------------------------------------------------------'

'Общая информация:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

'Три случайные строки:'

,id,f0,f1,f2,product
11938,j9ZbT,1.133416,-0.552499,2.910680,100.181451
49441,wUOLl,-0.919966,0.575044,3.984625,98.913478
21044,sZCfH,0.903704,0.536996,4.773943,107.672935


'-------------------------------------------------------------------------------------------------------------'

'Количество дубликатов:'

0

### Второе месторождение

In [20]:
information(region_two)

'Первые пять строк:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


'-------------------------------------------------------------------------------------------------------------'

'Общая информация:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

'Три случайные строки:'

,id,f0,f1,f2,product
97425,jz4JI,9.115242,1.992474,3.998383,107.813044
27301,KBbMs,10.381962,-1.616830,2.006880,53.906522
60996,fRsZk,-7.804705,-15.242453,4.993350,137.945408


'-------------------------------------------------------------------------------------------------------------'

'Количество дубликатов:'

0

### Третье месторождение:

In [21]:
information(region_three)

'Первые пять строк:'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


'-------------------------------------------------------------------------------------------------------------'

'Общая информация:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

'Три случайные строки:'

,id,f0,f1,f2,product
33978,g5JK7,-1.385587,-3.632214,1.065781,106.713904
16996,L6TAw,0.914602,2.507637,8.482196,154.659048
54790,Ynelq,-0.496467,0.085158,4.769798,27.772803


'-------------------------------------------------------------------------------------------------------------'

'Количество дубликатов:'

0

### Мини-вывод

Данные выведены на экран.Данные в норме.Дубликатов нет.Перейдем к обучению и проверке модели.

## Обучение и проверка модели

In [32]:
train_one, valid_one = train_test_split(region_one, test_size=0.25, random_state=12345)
train_two, valid_two = train_test_split(region_two, test_size=0.25, random_state=12345)
train_three, valid_three = train_test_split(region_three, test_size=0.25, random_state=12345)

In [50]:
features_train_one = train_one.drop(['id', 'product'], axis=1)
target_train_one = train_one['product']
features_valid_one = valid_one.drop(['id', 'product'], axis=1)
target_valid_one = valid_one['product']

features_train_two = train_two.drop(['id', 'product'], axis=1)
target_train_two = train_two['product']
features_valid_two = valid_two.drop(['id', 'product'], axis=1)
target_valid_two = valid_two['product']

features_train_three = train_three.drop(['id', 'product'], axis=1)
target_train_three = train_three['product']
features_valid_three = valid_three.drop(['id', 'product'], axis=1)
target_valid_three = valid_three['product']

In [51]:
display(f'Валидационная выборка первого датафрейма: {features_valid_one.shape}, {target_valid_one.shape}')
display(f'Валидационная выборка второго датафрейма: {features_valid_two.shape}, {target_valid_two.shape}')
display(f'Валидационная выборка третьего датафрейма: {features_valid_three.shape}, {target_valid_three.shape}')
display(f'Обучающая выборка первого датафрейма: {features_train_one.shape}, {target_train_one.shape}')
display(f'Обучающая выборка второго датафрейма: {features_train_two.shape}, {target_train_two.shape}')
display(f'Обучающая выборка третьего датафрейма: {features_train_three.shape}, {target_train_three.shape}')

'Валидационная выборка первого датафрейма: (25000, 3), (25000,)'

'Валидационная выборка второго датафрейма: (25000, 3), (25000,)'

'Валидационная выборка третьего датафрейма: (25000, 3), (25000,)'

'Обучающая выборка первого датафрейма: (75000, 3), (75000,)'

'Обучающая выборка второго датафрейма: (75000, 3), (75000,)'

'Обучающая выборка третьего датафрейма: (75000, 3), (75000,)'

### Первое месторождение:

In [71]:
model = LinearRegression()
model.fit(features_train_one, target_train_one)
predicted_valid_one = model.predict(features_valid_one)
predicted_valid_one_mean = predicted_valid_one.mean()
target_valid_one_mean = target_valid_one.mean()
mse_one = mean_squared_error(target_valid_one, predicted_valid_one)
display(f'RMSE: {mse_one ** 0.5}')
display(f'Cредний запас предсказанного сырья: {predicted_valid_one_mean}')
display(f'Средний фактический запас сырья: {target_valid_one_mean}')

'RMSE: 37.5794217150813'

'Cредний запас предсказанного сырья: 92.59256778438035'

'Средний фактический запас сырья: 92.07859674082927'

### Второе месторождение:

In [70]:
model = LinearRegression()
model.fit(features_train_two, target_train_two)
predicted_valid_two = model.predict(features_valid_two)
predicted_valid_two_mean = predicted_valid_two.mean()
target_valid_two_mean = target_valid_two.mean()
mse_two = mean_squared_error(target_valid_two, predicted_valid_two)
display(f'RMSE : {mse_two ** 0.5}')
display(f'Cредний запас предсказанного сырья: {predicted_valid_two_mean}')
display(f'Средний фактический запас сырья: {target_valid_two_mean}')

'RMSE : 0.893099286775617'

'Cредний запас предсказанного сырья: 68.728546895446'

'Средний фактический запас сырья: 68.72313602435997'

### Третье месторождение:

In [69]:
model = LinearRegression()
model.fit(features_train_three, target_train_three)
predicted_valid_three = model.predict(features_valid_three)
predicted_valid_three_mean = predicted_valid_three.mean()
target_valid_three_mean = target_valid_three.mean()
mse_three = mean_squared_error(target_valid_three, predicted_valid_three)
display(f'RMSE: {mse_three ** 0.5}')
display(f'Cредний запас предсказанного сырья: {predicted_valid_three_mean}')
display(f'Средний фактический запас сырья: {target_valid_three_mean}')

'RMSE: 0.893099286775617'

'Cредний запас предсказанного сырья: 94.96504596800489'

'Средний фактический запас сырья: 94.88423280885438'

### Мини-вывод:

Данные выведены на экран.Сначала мы разделили данные в нужной нам пропорции.Позже узнали размер выборок.А затем уже прошлись по трем датафреймам и узнали три значения RMSE и средние запасы предсказанного и фактичего сырья.Значение RMSE у первого месторождения огромное,модель показывает себя не очень.Значения RMSE у второго и третьего месторождения одинаковы и очень малы, модели показывают себя хорошо.Средний запас предсказанного и фактического сырья у третьего месторожения наилучший.Перейдем к расчету прибыли.

## Подготовка к расчёту прибыли

In [73]:
Budget = 10000000000 / 200
Income = 450000
Payback = budget / income
display(f'Бюджет для разработки одной скважины: {Budget}')
display(f'Доход с каждой единицы продукта: {Income}')
display(f'Cколько нужно тысяч баррелей для того, чтобы окупить затраты: {Payback}')
display(f'Объем скважины для безубыточной разработки в 1 месторождении: {Payback - region_one["product"].mean()}')
display(f'Объем скважины для безубыточной разработки в 2 месторождении: {Payback - region_two["product"].mean()}')
display(f'Объем скважины для безубыточной разработки в 3 месторождении: {Payback - region_three["product"].mean()}')

'Бюджет для разработки одной скважины: 50000000.0'

'Доход с каждой единицы продукта: 450000'

'Cколько нужно тысяч баррелей для того, чтобы окупить затраты: 111.11111111111111'

'Объем скважины для безубыточной разработки в 1 месторождении: 18.6111111111111'

'Объем скважины для безубыточной разработки в 2 месторождении: 42.2861111111111'

'Объем скважины для безубыточной разработки в 3 месторождении: 16.11111111111107'

### Мини-вывод:
Разработка новых скважин во 2 месторождении является наиболее убыточной, как по мне,т.к для нее требуется наибольший объем. Объем скважин для безубыточной разработки в 1 и 3 месторождениях примерно равны. Наименьший объем требуется в 3 месторождении. Это связано с тем, что среднее значение в 3 месторождении наибольшее.

## Расчёт прибыли и рисков 

In [94]:
def profit_calculation(target, predictions, count):
    profit = 0
    predictions_sorted = predictions.sort_values(ascending=False).head(500)
    target_sorted = target[predictions_sorted.index][:count]
    for barrels in target_sorted:
        profit += (barrels - payback) * income
    return profit / 1000000

In [96]:
valid_one['product_predicted'] = predicted_valid_one
valid_two['product_predicted'] = predicted_valid_two
valid_three['product_predicted'] = predicted_valid_three
display(f'Прибыльность лучших 200 точек в первом месторождении: {round(profit_calculation(valid_one["product"], valid_one["product_predicted"], 200))} миллионов')
display(f'Прибыльность лучших 200 точек во втором месторождении: {round(profit_calculation(valid_two["product"], valid_two["product_predicted"], 200))} миллионов')
display(f'Прибыльность лучших 200 точек в третьем месторождении: {round(profit_calculation(valid_three["product"], valid_three["product_predicted"], 200))} миллионов')

'Прибыльность лучших 200 точек в первом месторождении: 3321 миллионов'

'Прибыльность лучших 200 точек во втором месторождении: 2415 миллионов'

'Прибыльность лучших 200 точек в третьем месторождении: 2710 миллионов'

### Мини-вывод:
Предположение, что разработка новых скважин во 2 месторождении является наиболее убыточной подтвердилось.Наиболее прибыльным является разработка в первом месторождении.Потенциальная прибыль на 600 млн больше,чем в третьем месторождении,и почти на миллиард больше,чем во втором месторождении.

In [112]:
def risks(target, predictions):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        values.append(profit_calculation(target_subsample, preds_subsample, 200))
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    display(f'Вероятность убытков в месторождении: {stats.percentileofscore(values, 0)} %') 
    display(f'Средняя прибыль: {values.mean()} миллионов')
    display(f'Нижняя граница доверительного значения: {lower}')
    display(f'Верхняя граница доверительного значения: {upper}')

### Первое месторождение:

In [113]:
risks(valid_one['product'], valid_one['product_predicted'])

'Вероятность убытков в месторождении: 6.0 %'

'Средняя прибыль: 425.9385269105921 миллионов'

'Нижняя граница доверительного значения: -102.09009483793763'

'Верхняя граница доверительного значения: 947.976353358368'

### Второе месторождение:

In [114]:
risks(valid_two['product'], valid_two['product_predicted'])

'Вероятность убытков в месторождении: 1.0 %'

'Средняя прибыль: 515.2227734432944 миллионов'

'Нижняя граница доверительного значения: 68.87322537050716'

'Верхняя граница доверительного значения: 931.5475912570474'

### Третье месторождение:

In [115]:
risks(valid_three['product'], valid_three['product_predicted'])

'Вероятность убытков в месторождении: 6.4 %'

'Средняя прибыль: 435.0083627827554 миллионов'

'Нижняя граница доверительного значения: -128.88054732978947'

'Верхняя граница доверительного значения: 969.7069541802659'

# Вывод

Судя по полученным данным, лучшим решением будет бурить скважины во втором месторождение.Именно во втором месторождение вероятность убытков наименьшая,а именно 1%.Также там наибольшая средняя прибыль среди всех трех месторождений.